<a href="https://colab.research.google.com/github/himoyuzuki/colab_analytics/blob/main/%E6%97%A5%E6%9C%AC%E3%81%AE%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E5%88%A5%E4%BA%BA%E5%8F%A3%E3%82%92%E5%8F%AF%E8%A6%96%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本の都道府県別人口を可視化

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
import io
import folium

In [ ]:
# データの読み込み
# 出典: 国勢調査 時系列データ CSV形式による主要時系列データ | ファイル | 統計データを探す | 政府統計の総合窓口
url = 'https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031524010&fileKind=1'
res = requests.get(url).content
df1 = pd.read_csv(io.StringIO(res.decode('shift-jis')), header=0)
df1.head()

,都道府県コード,都道府県名,元号,和暦（年）,西暦（年）,注,人口（総数）,人口（男）,人口（女）
0,00,全国,大正,9.0,1920.0,NaN,55963053,28044185,27918868
1,01,北海道,大正,9.0,1920.0,NaN,2359183,1244322,1114861
2,02,青森県,大正,9.0,1920.0,NaN,756454,381293,375161
3,03,岩手県,大正,9.0,1920.0,NaN,845540,421069,424471
4,04,宮城県,大正,9.0,1920.0,NaN,961768,485309,476459


In [ ]:
# 出典: land/prefecturalCapital.csv at master · dataofjapan/land · GitHub
url = 'https://raw.githubusercontent.com/dataofjapan/land/master/prefecturalCapital.csv'
df2 = pd.read_csv(url)
df2.head()

,id,nam_ja,lat,lon
0,1,北海道,43.064359,141.347449
1,2,青森,40.824294,140.740054
2,3,岩手,39.703530,141.152667
3,4,宮城,38.268737,140.872183
4,5,秋田,39.718175,140.103356


In [ ]:
# データ抽出
# 1年分の、都道府県ごとの人口だけのDataFrameにする
population_df = df1[df1['西暦（年）'] == 2015.0]
population_df = population_df.rename(columns={'都道府県名' : '都道府県', '人口（総数）' : '人口'})
population_df['人口'] = population_df['人口'].astype(int)
population_df = population_df.reset_index()[['都道府県', '人口']].set_index('都道府県')
population_df = population_df.drop(index=['全国', '人口集中地区', '人口集中地区以外の地区'])
population_df.head()

,人口
都道府県,
北海道,5381733
青森県,1308265
岩手県,1279594
宮城県,2333899
秋田県,1023119


In [ ]:
# 都道府県名の表記揺れを統一させる
geo_df = df2.copy()
geo_df['nam_ja'] = geo_df['nam_ja'].apply(lambda x: x if x[-1] == '京' or x[-1] == '道' or x[-1] == '都' or x[-1] == '阪' else x + '県')
geo_df['nam_ja'] = geo_df['nam_ja'].apply(lambda x: x + '府' if x == '大阪' or x == '京都' else x)
geo_df['nam_ja'] = geo_df['nam_ja'].apply(lambda x: x + '都' if x == '東京' else x)
geo_df = geo_df.rename(columns={'nam_ja' : '都道府県'})
geo_df = geo_df.reset_index()[['都道府県', 'lat', 'lon']].set_index('都道府県')
geo_df.head()

,lat,lon
都道府県,,
北海道,43.064359,141.347449
青森県,40.824294,140.740054
岩手県,39.703530,141.152667
宮城県,38.268737,140.872183
秋田県,39.718175,140.103356


In [ ]:
# グラフ描画するデータを1つのDataFrameに結合
output_df = pd.merge(population_df, geo_df, how='outer', left_index=True, right_index=True)
output_df

,人口,lat,lon
都道府県,,,
三重県,1815865,34.730547,136.508610
京都府,2610353,35.020996,135.753114
佐賀県,832832,33.249367,130.298822
兵庫県,5534800,34.691280,135.183087
北海道,5381733,43.064359,141.347449
千葉県,6222666,35.604563,140.123179
和歌山県,963579,34.224806,135.167950
埼玉県,7266534,35.857771,139.647804
大分県,1166338,33.238200,131.612674


In [ ]:
location = output_df[output_df.index == '東京都'][['lat', 'lon']]

m = folium.Map(location=location, zoom_start=5)
for index, row in output_df.iterrows():
  folium.Circle(
    location=row[['lat', 'lon']],
    radius=np.sqrt(row['人口']) * 20,
    color='Blue',
    fill=True,
    weight=0
  ).add_to(m)
m

/usr/local/lib/python3.11/dist-packages/folium/utilities.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])
/usr/local/lib/python3.11/dist-packages/folium/utilities.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])
/usr/local/lib/python3.11/dist-packages/folium/utilities.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1]